<a href="https://colab.research.google.com/github/steelpipe75/kagglebook-for-colab/blob/master/ch03/ch03-05-reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ---------------------------------
# データ等の準備
# ----------------------------------
import numpy as np
import pandas as pd

In [2]:
import importlib
import sys
import subprocess

# Google Colab 上で実行しているかどうかを判断するフラグ
ON_COLAB = "google.colab" in sys.modules
print(f"ON_COLAB: {ON_COLAB}")

if ON_COLAB:
    USE_GIT = True # Gitを使う
    # USE_GIT = False # Gitを使わない

    print(f"USE_GIT: {USE_GIT}")
    if USE_GIT:
        !git clone https://github.com/ghmagazine/kagglebook.git
    else:
        # Google Drive にマウントする
        drive = importlib.import_module("google.colab.drive")
        drive.mount("/content/drive/")

        import os
        colab_dir = "/content/drive/MyDrive/kagglebook/" # データ置き場

    # 必要なライブラリをインストール
    %pip install bhtsne

ON_COLAB: True
USE_GIT: True
Cloning into 'kagglebook'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 95 (delta 19), reused 87 (delta 18), pack-reused 1 (from 1)
Receiving objects: 100% (95/95), 3.13 MiB | 11.35 MiB/s, done.
Resolving deltas: 100% (19/19), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bhtsne: filename=bhtsne-0.1.9-cp311-cp311-linux_x86_64.whl size=280156 sha256=eae3fa4376c4f827eab50103ffdd86323c8bdb630654867c05db4550a29bb729
  Stored in directory: /root/.cache/pip/wheels/c7/e0/05/ef25429ca93e3ccaddddb1ed036fb692b6b99f078b83937842
Successfully built bhtsne


train_xは学習データ、train_yは目的変数、test_xはテストデータ
pandasのDataFrame, Seriesで保持します。（numpyのarrayで保持することもあります）

In [3]:
if ON_COLAB:
    if USE_GIT:
        train = pd.read_csv('/content/kagglebook/input/sample-data/train_preprocessed_onehot.csv')
    else:
        train = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/train_preprocessed_onehot.csv'))
else:
    train = pd.read_csv('../input/sample-data/train_preprocessed_onehot.csv')
train_x = train.drop(['target'], axis=1)
train_y = train['target']
if ON_COLAB:
    if USE_GIT:
        test_x = pd.read_csv('/content/kagglebook/input/sample-data/test_preprocessed_onehot.csv')
    else:
        test_x = pd.read_csv(os.path.join(colab_dir, 'input/sample-data/test_preprocessed_onehot.csv'))
else:
    test_x = pd.read_csv('../input/sample-data/test_preprocessed_onehot.csv')

In [4]:
# 説明用に学習データとテストデータの元の状態を保存しておく
train_x_saved = train_x.copy()
test_x_saved = test_x.copy()

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [6]:
# 標準化を行った学習データとテストデータを返す関数
def load_standarized_data():
    train_x, test_x = train_x_saved.copy(), test_x_saved.copy()

    scaler = StandardScaler()
    scaler.fit(train_x)
    train_x = scaler.transform(train_x)
    test_x = scaler.transform(test_x)
    return pd.DataFrame(train_x), pd.DataFrame(test_x)

In [7]:
# MinMaxスケーリングを行った学習データとテストデータを返す関数
def load_minmax_scaled_data():
    train_x, test_x = train_x_saved.copy(), test_x_saved.copy()

    # Min-Max Scalingを行う
    scaler = MinMaxScaler()
    scaler.fit(pd.concat([train_x, test_x], axis=0))
    train_x = scaler.transform(train_x)
    test_x = scaler.transform(test_x)

    return pd.DataFrame(train_x), pd.DataFrame(test_x)

In [8]:
# -----------------------------------
# PCA
# -----------------------------------
# 標準化されたデータを用いる
train_x, test_x = load_standarized_data()
# -----------------------------------
# PCA
from sklearn.decomposition import PCA

データは標準化などのスケールを揃える前処理が行われているものとする

In [9]:
# 学習データに基づいてPCAによる変換を定義
pca = PCA(n_components=5)
pca.fit(train_x)

PCA(n_components=5)

In [10]:
# 変換の適用
train_x = pca.transform(train_x)
test_x = pca.transform(test_x)

In [11]:
# -----------------------------------
# 標準化されたデータを用いる
train_x, test_x = load_standarized_data()
# -----------------------------------
# TruncatedSVD
from sklearn.decomposition import TruncatedSVD

データは標準化などのスケールを揃える前処理が行われているものとする

In [12]:
# 学習データに基づいてSVDによる変換を定義
svd = TruncatedSVD(n_components=5, random_state=71)
svd.fit(train_x)

TruncatedSVD(n_components=5, random_state=71)

In [13]:
# 変換の適用
train_x = svd.transform(train_x)
test_x = svd.transform(test_x)

In [14]:
# -----------------------------------
# NMF
# -----------------------------------
# 非負の値とするため、MinMaxスケーリングを行ったデータを用いる
train_x, test_x = load_minmax_scaled_data()
# -----------------------------------
from sklearn.decomposition import NMF

データは非負の値から構成されているとする

In [15]:
# 学習データに基づいてNMFによる変換を定義
model = NMF(n_components=5, init='random', random_state=71)
model.fit(train_x)

NMF(init='random', n_components=5, random_state=71)

In [16]:
# 変換の適用
train_x = model.transform(train_x)
test_x = model.transform(test_x)

In [17]:
# -----------------------------------
# LatentDirichletAllocation
# -----------------------------------
# MinMaxスケーリングを行ったデータを用いる
# カウント行列ではないが、非負の値であれば計算は可能
train_x, test_x = load_minmax_scaled_data()
# -----------------------------------
from sklearn.decomposition import LatentDirichletAllocation

データは単語文書のカウント行列などとする

In [18]:
# 学習データに基づいてLDAによる変換を定義
model = LatentDirichletAllocation(n_components=5, random_state=71)
model.fit(train_x)

LatentDirichletAllocation(n_components=5, random_state=71)

In [19]:
# 変換の適用
train_x = model.transform(train_x)
test_x = model.transform(test_x)

In [20]:
# -----------------------------------
# LinearDiscriminantAnalysis
# -----------------------------------
# 標準化されたデータを用いる
train_x, test_x = load_standarized_data()
# -----------------------------------
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

データは標準化などのスケールを揃える前処理が行われているものとする

In [21]:
# 学習データに基づいてLDAによる変換を定義
lda = LDA(n_components=1)
lda.fit(train_x, train_y)

LinearDiscriminantAnalysis(n_components=1)

In [22]:
# 変換の適用
train_x = lda.transform(train_x)
test_x = lda.transform(test_x)

In [23]:
# -----------------------------------
# t-sne
# -----------------------------------
# 標準化されたデータを用いる
train_x, test_x = load_standarized_data()
# -----------------------------------
import bhtsne

データは標準化などのスケールを揃える前処理が行われているものとする

In [24]:
# t-sneによる変換
data = pd.concat([train_x, test_x])
embedded = bhtsne.tsne(data.astype(np.float64), dimensions=2, rand_seed=71)

In [25]:
# -----------------------------------
# UMAP
# -----------------------------------
# 標準化されたデータを用いる
train_x, test_x = load_standarized_data()
# -----------------------------------
import umap

データは標準化などのスケールを揃える前処理が行われているものとする

In [26]:
# 学習データに基づいてUMAPによる変換を定義
um = umap.UMAP()
um.fit(train_x)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


UMAP(tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True})

In [27]:
# 変換の適用
train_x = um.transform(train_x)
test_x = um.transform(test_x)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [28]:
# -----------------------------------
# クラスタリング
# -----------------------------------
# 標準化されたデータを用いる
train_x, test_x = load_standarized_data()
# -----------------------------------
from sklearn.cluster import MiniBatchKMeans

データは標準化などのスケールを揃える前処理が行われているものとする

In [29]:
# 学習データに基づいてMini-Batch K-Meansによる変換を定義
kmeans = MiniBatchKMeans(n_clusters=10, random_state=71)
kmeans.fit(train_x)

MiniBatchKMeans(n_clusters=10, random_state=71)

In [30]:
# 属するクラスタを出力する
train_clusters = kmeans.predict(train_x)
test_clusters = kmeans.predict(test_x)

In [31]:
# 各クラスタの中心までの距離を出力する
train_distances = kmeans.transform(train_x)
test_distances = kmeans.transform(test_x)